In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
from tqdm import tqdm
import numpy as np
import time
import os
import matplotlib.pyplot as plt
from collections import defaultdict

import color_regions, network, visualizations, utils
from color_regions import *
from network import *
from visualizations import *
from utils import *

torch.backends.cudnn.benchmark = True
device = "cuda:0" if torch.cuda.is_available() else "cpu"

In [ ]:
# set up autoreloading of shared code
%load_ext autoreload
%autoreload 1
%aimport color_regions,network,visualizations,utils
%aimport

In [ ]:
import sys
prev_time = 0
gamma = 0.99
stats = {}  # tracks ewma running average
def benchmark(point=None, profile=True, verbose=True, cuda=True): # not thread safe at all
    global prev_time
    if not profile:
        return
    if cuda:
        torch.cuda.synchronize()
    time_now = time.perf_counter()
    if point is not None:
        point = f"{sys._getframe().f_back.f_code.co_name}-{point}"
        time_taken = time_now - prev_time
        if point not in stats:
            stats[point] = time_taken
        stats[point] = stats[point]*gamma + time_taken*(1-gamma)
        if verbose:
            print(f"took {time_taken} to reach {point}, ewma={stats[point]}")
    prev_time = time_now

In [ ]:
main_data_set = TextureDatasetGenerator("./data/dtd")  # -> do this so we only load once

In [ ]:
transform = transforms.Compose(
     [transforms.ToTensor()])
      #transforms.RandomRotation(90)])

batch_size = 128  # seems to be the fastest batch size
train_indices = (0, 500_000) # size of training set
valid_indices = (1_250_000, 1_260_000)
test_indices = (2_260_000, 2_270_000)

def set_loader_helper(indices):
    data_set = TextureDatasetGenerator(main_data_set,
                                       transform=transform,
                                       noise_size=(5,15),
                                       size=128,
                                       radius_frac=(1/3, 1/2.1),
                                       image_indices=indices)
    loader = torch.utils.data.DataLoader(data_set, batch_size=batch_size, 
                                          shuffle=True, num_workers=4, pin_memory=True)
    return data_set, loader
train_set, train_loader = set_loader_helper(train_indices)
valid_set, valid_loader = set_loader_helper(valid_indices)
test_set, test_loader = set_loader_helper(test_indices)

In [ ]:
res_net = ResNet([[32, 7, 2],  # num_channels (input and output), kernel_size, stride
                  [64, 3, 1],
                  [64, 3, 1],
                  [128, 3, 2],
                  [128, 3, 1],
                  [128, 3, 1],
                  [256, 3, 2],
                  [256, 3, 1],
                  [512, 3, 2],
                  [512, 3, 1]], valid_set.num_classes, [128, 128, 3], 
                   "texture_net.dict", fc_layers=[160]).to(device)

loss_func = nn.CrossEntropyLoss()
optim = torch.optim.Adam(res_net.parameters())
print(res_net.num_params())
res_net.load_model_state_dict(optim=optim)

In [ ]:
results = train(res_net, optim, loss_func, 200)

In [ ]:
np.random.seed(500_001)
explain_img, explain_target_logit, *__ = valid_set.generate_one()
heat_map = finite_differences_map(res_net, valid_set, explain_target_logit.argmax(), explain_img, device=device, batch_size=127)

In [ ]:
plt.subplot(1, 2, 1)
plt.imshow(explain_img/255.)
plt.subplot(1, 2, 2)
imshow_centered_colorbar(heat_map.sum(axis=2), cmap="bwr", title="FD Map")
print(valid_set.idx_to_texname[explain_target_logit.argmax()])

In [ ]:
default_scales = [3,5,7,9,13,15]
if 1: 
    %store -r pca_directions_1_stride pca_directions_s_stride
else:
    pca_directions_1_stride = find_pca_directions(valid_set, 16384, default_scales, 1)
    pca_directions_s_stride = find_pca_directions(valid_set, 16384, default_scales, default_scales)
    %store pca_directions_1_stride pca_directions_s_stride

In [ ]:
visualize_pca_directions(pca_directions_1_stride, "Strides=1", default_scales, component=0, lines=False)

In [ ]:
visualize_pca_directions(pca_directions_1_stride, "Strides=1", default_scales, component=1, lines=False)

In [ ]:
visualize_pca_directions(pca_directions_1_stride, "Strides=1", default_scales, component=2, lines=False)

In [ ]:
visualize_pca_directions(pca_directions_1_stride, "Strides=1", default_scales, component=3, lines=False)

In [ ]:
visualize_pca_directions(pca_directions_s_stride, "Strides=scales", default_scales, component=0, lines=True)

In [ ]:
visualize_pca_directions(pca_directions_s_stride, "Strides=scales", default_scales, component=1, lines=True)

In [ ]:
visualize_pca_directions(pca_directions_s_stride, "Strides=scales", default_scales, component=2, lines=True)

In [ ]:
visualize_pca_directions(pca_directions_s_stride, "Strides=scales", default_scales, component=3, lines=True)

In [ ]:
np.random.seed(500_001)
explain_img, explain_target_logit, *__ = valid_set.generate_one()
result = pca_direction_grids(res_net, valid_set, explain_target_logit.argmax(), explain_img, default_scales, 
                    pca_directions_s_stride, device=device, batch_size=128)

In [ ]:
plt.subplot(1,4,1)
plt.imshow(explain_img.squeeze())
for c in range(3):
    plt.subplot(1,4,c+2)
    imshow_centered_colorbar(result[...,c], cmap="bwr", title=f"Channel {c}")

In [ ]:
seeds = [1_2123, 1_40_124, 1_508_559, 1_5_019_258, 1_2_429_852, 9032, 5832, 12, 5014, 92, 42, 52, 
         52_934, 935_152, 1_000_000, 1_000_001, 27, 24, 512, 999_105]  # 20 
def generate_many_pca(net, component=0):
    _pca_map_s_strides = []
    _pca_map_1_strides = []
    _grad_maps = []
    _explain_imgs = []
    for seed in seeds:
        np.random.seed(seed)
        generated_img, label, *__ = valid_set.generate_one()
        tensored_img = tensorize(generated_img, device=device, requires_grad=True)
        grad_map = torch.autograd.grad(net(tensored_img)[0,label.argmax()], tensored_img)[0]
        pca_map_strided = pca_direction_grids(net, valid_set, label.argmax(), generated_img, 
                                              default_scales, pca_directions_s_stride, 
                                              device=device, batch_size=128, component=component)
        pca_map_1_stride = pca_direction_grids(net, valid_set, label.argmax(), generated_img, 
                                              default_scales, pca_directions_1_stride, component=component 
                                              device=device, batch_size=128, strides=1)
        _explain_imgs.append(generated_img)
        _grad_maps.append(grad_map.detach().cpu().squeeze(0).numpy().transpose(1,2,0))
        _pca_map_s_strides.append(pca_map_strided.copy())
        _pca_map_1_strides.append(pca_map_1_stride.copy())
    return _pca_map_s_strides, _pca_map_1_strides, _grad_maps, _explain_imgs

In [ ]:
pca_map_s_strides, pca_map_1_strides, grad_maps, explain_imgs = generate_many_pca(res_net)

In [ ]:
texture_pca_map_1_strides_2023_01_29 = pca_map_1_strides
%store texture_pca_map_1_strides_2023_01_29

In [ ]:
grid_figure_list = []
for i, img in enumerate(pca_map_1_strides):
    grid_figure_row = [explain_imgs[i]]
    grid_figure_row += [x for x in img.transpose(2,0,1)]
    grid_figure_row += [b for b in grad_maps[i].transpose(2,0,1)]
    grid_figure_list.append(grid_figure_row)
plt_grid_figure(grid_figure_list, titles=["Image", 
                                          "Channel 0 PCA (strides=1)",
                                          "Channel 1 PCA (strides=1)",
                                          "Channel 2 PCA (strides=1)",
                                          "Channel 0 Gradient Map",
                                          "Channel 1 Gradient Map",
                                          "Channel 2 Gradient Map"], cmap="bwr", transpose=False)
    

# PCA Direction convergence experiments

In [ ]:
visualize_pca_directions(pca_directions_1_stride, "Strides=1", default_scales)

In [ ]:
visualize_pca_directions(pca_directions_s_stride, "Strides=scales", default_scales)

In [ ]:
default_scales = [3,5,7,9,13,15]
small_pca_directions_1_stride = find_pca_directions(valid_set, 512, default_scales, 1)
small_pca_directions_s_stride = find_pca_directions(valid_set, 512, default_scales, default_scales)

In [ ]:
visualize_pca_directions(small_pca_directions_1_stride, "Strides=1", default_scales)

In [ ]:
visualize_pca_directions(small_pca_directions_s_stride, "Strides=scales", default_scales)

In [ ]:
np.random.seed(510)
test_directions = find_pca_directions(valid_set, 8192*4, default_scales, default_scales, component=0)

In [ ]:
visualize_pca_directions(test_directions, "Strides=scales", default_scales) 
# component 0
# seed 510 gargantuan (32768) sample

In [ ]:
visualize_pca_directions(test_directions, "Strides=scales", default_scales)
# component 1
# seed 510 gargantuan (32768) sample

In [ ]:
visualize_pca_directions(test_directions, "Strides=scales", default_scales)
# component 1
# seed 507 gargantuan (32768) sample

In [ ]:
visualize_pca_directions(test_directions, "Strides=scales", default_scales)
# component 1
# seed 507, huge (8192) sample

In [ ]:
visualize_pca_directions(test_directions, "Strides=scales", default_scales) 
# component 1
# seed 507, large (2048) sample

In [ ]:
visualize_pca_directions(test_directions, "Strides=scales", default_scales) 
# component 1
# seed 508, large (2048) sample

In [ ]:
visualize_pca_directions(test_directions, "Strides=scales", default_scales)
# component 1
# seed 508, small sample

In [ ]:
visualize_pca_directions(test_directions, "Strides=scales", default_scales) 
# component 1
# seed 507, small sample

What if we run the same experiment, but cheat with a prior on pixel values that we know *should* be informative to the output logit, namely values closest to the decision boundary?

# Model Optimization Stuff

In [ ]:
res_net.save_model_state_dict(optim=optim)

In [ ]:
for _ in range(1000):
    res_net.forward(generated_img, profile=True)

In [ ]:
total = sum(stats.values())  # --> gave 3x speed! (Fast and Accurate Model scaling?)
for k,v in stats.items():    # --> the 3x speedup caused underfitting though, so switched to 2x
    print(k,(100.*v/total))
